In [1]:
# mlp for multi-output regression
from numpy import mean
import pandas as pd
import numpy as np
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import BatchNormalization, Activation, Dropout, Flatten, LayerNormalization
from tensorflow.keras.optimizers import Adam


In [ ]:
#     dataset = tf.data.Dataset.from_tensor_slices(mdf[features+targets].values)    
#     #dataset = tf.data.Dataset.from_tensor_slices(df3[features+targets].values)

#     # Window the data but only take those with the specified size
#     dataset = dataset.window(5, shift=1, drop_remainder=True)

#     # # Flatten the windows by putting its elements in a single batch
#     dataset = dataset.flat_map(lambda window: window.batch(5))
#     # # Create tuples with features (first four elements of the window) and labels (last element)
#     dataset = dataset.map(lambda window: (tf.reshape(window[:-1], [-1]), window[-1][-2:]))

In [2]:
x = range(200)
x = np.asarray(x).reshape(20, 10)
print(x)
# np.concatenate((x, x), axis=1)

# y = tf.data.Dataset.from_tensor_slices(x)
# #y[:, -3:] = 10 - y[:, -3:]
# list(y.as_numpy_iterator())

y = tf.convert_to_tensor(x)
# x = [1, 2, 3, 4, 5]
# y = tf.constant([5, 4, 3, 2, 1])
tf.concat([y[:, :-3],
           tf.subtract(y[-1][-1], y[:, -3:])], 1)


[[  0   1   2   3   4   5   6   7   8   9]
 [ 10  11  12  13  14  15  16  17  18  19]
 [ 20  21  22  23  24  25  26  27  28  29]
 [ 30  31  32  33  34  35  36  37  38  39]
 [ 40  41  42  43  44  45  46  47  48  49]
 [ 50  51  52  53  54  55  56  57  58  59]
 [ 60  61  62  63  64  65  66  67  68  69]
 [ 70  71  72  73  74  75  76  77  78  79]
 [ 80  81  82  83  84  85  86  87  88  89]
 [ 90  91  92  93  94  95  96  97  98  99]
 [100 101 102 103 104 105 106 107 108 109]
 [110 111 112 113 114 115 116 117 118 119]
 [120 121 122 123 124 125 126 127 128 129]
 [130 131 132 133 134 135 136 137 138 139]
 [140 141 142 143 144 145 146 147 148 149]
 [150 151 152 153 154 155 156 157 158 159]
 [160 161 162 163 164 165 166 167 168 169]
 [170 171 172 173 174 175 176 177 178 179]
 [180 181 182 183 184 185 186 187 188 189]
 [190 191 192 193 194 195 196 197 198 199]]


<tf.Tensor: shape=(20, 10), dtype=int32, numpy=
array([[  0,   1,   2,   3,   4,   5,   6, 192, 191, 190],
       [ 10,  11,  12,  13,  14,  15,  16, 182, 181, 180],
       [ 20,  21,  22,  23,  24,  25,  26, 172, 171, 170],
       [ 30,  31,  32,  33,  34,  35,  36, 162, 161, 160],
       [ 40,  41,  42,  43,  44,  45,  46, 152, 151, 150],
       [ 50,  51,  52,  53,  54,  55,  56, 142, 141, 140],
       [ 60,  61,  62,  63,  64,  65,  66, 132, 131, 130],
       [ 70,  71,  72,  73,  74,  75,  76, 122, 121, 120],
       [ 80,  81,  82,  83,  84,  85,  86, 112, 111, 110],
       [ 90,  91,  92,  93,  94,  95,  96, 102, 101, 100],
       [100, 101, 102, 103, 104, 105, 106,  92,  91,  90],
       [110, 111, 112, 113, 114, 115, 116,  82,  81,  80],
       [120, 121, 122, 123, 124, 125, 126,  72,  71,  70],
       [130, 131, 132, 133, 134, 135, 136,  62,  61,  60],
       [140, 141, 142, 143, 144, 145, 146,  52,  51,  50],
       [150, 151, 152, 153, 154, 155, 156,  42,  41,  40],
       [

In [3]:
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.window(3, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(3))

def foo(x):
    #x[:, -2:] = x[-1][-1] - x[:, -2:]
    x = tf.concat([x[:, :-2], tf.subtract(x[-1][-1], x[:, -2:])], 1)
    return x
dataset = dataset.map(lambda x: (foo(x[:,:-1]), x[:, -1][-1]))

for i,j in dataset:
    print(i.numpy())
    print(j.numpy())

[[ 0  1  2  3  4  5  6 21 20]
 [10 11 12 13 14 15 16 11 10]
 [20 21 22 23 24 25 26  1  0]]
29
[[10 11 12 13 14 15 16 21 20]
 [20 21 22 23 24 25 26 11 10]
 [30 31 32 33 34 35 36  1  0]]
39
[[20 21 22 23 24 25 26 21 20]
 [30 31 32 33 34 35 36 11 10]
 [40 41 42 43 44 45 46  1  0]]
49
[[30 31 32 33 34 35 36 21 20]
 [40 41 42 43 44 45 46 11 10]
 [50 51 52 53 54 55 56  1  0]]
59
[[40 41 42 43 44 45 46 21 20]
 [50 51 52 53 54 55 56 11 10]
 [60 61 62 63 64 65 66  1  0]]
69
[[50 51 52 53 54 55 56 21 20]
 [60 61 62 63 64 65 66 11 10]
 [70 71 72 73 74 75 76  1  0]]
79
[[60 61 62 63 64 65 66 21 20]
 [70 71 72 73 74 75 76 11 10]
 [80 81 82 83 84 85 86  1  0]]
89
[[70 71 72 73 74 75 76 21 20]
 [80 81 82 83 84 85 86 11 10]
 [90 91 92 93 94 95 96  1  0]]
99
[[ 80  81  82  83  84  85  86  21  20]
 [ 90  91  92  93  94  95  96  11  10]
 [100 101 102 103 104 105 106   1   0]]
109
[[ 90  91  92  93  94  95  96  21  20]
 [100 101 102 103 104 105 106  11  10]
 [110 111 112 113 114 115 116   1   0]]
119
[[10

In [88]:
# x = tf.convert_to_tensor(range(100))
window_size = 10
x = range(100)
# x = np.asarray(x).reshape(10, -1)
print(x)
print('----------------------')
x = tf.data.Dataset.from_tensor_slices(x) # Creates a Dataset whose elements are slices of the given tensors.
print(list(x.as_numpy_iterator()))
print()

# x1 = x.map(lambda x: (x[:-1], x[-1]))
# list(x1.as_numpy_iterator())

x = x.window(window_size+1, shift=1, drop_remainder=True)
for k in x:
    print(list(k.as_numpy_iterator()), '<- first window, because we break')
    break

# x = x.flat_map(lambda x: x.batch(window_size+1))
# print(list(x.as_numpy_iterator())[:2], '<- first two window after batch with 9')

x = x.flat_map(lambda w: w.batch(window_size+1))
x = x.map(lambda w: (w[:-1], w[-1]))

print(list(x.as_numpy_iterator())[:2])
print('-' * 80)
x = x.shuffle(buffer_size=10)
print(list(x.as_numpy_iterator())[:2])


x = x.batch(5)
print('-' * 80, 'batch')
print(list(x.as_numpy_iterator())[:2])


x = x.prefetch(1)
print('-' * 80, 'prefetch')
print(list(x.as_numpy_iterator())[:2])

range(0, 100)
----------------------
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] <- first window, because we break
[(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 10), (array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), 11)]
--------------------------------------------------------------------------------
[(array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17]), 18), (array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19]), 20)]
-------------------------------------------------------------------------------- batch
[(array([[ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       [ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 6,  7,  8,  9

In [ ]:
# x1 = x.flat_map(lambda x: x.batch(window_size+1))
# print(list(x1.as_numpy_iterator())[:2], '<- first two window after batch with 8')
    
x = x.flat_map(lambda x: x.batch(window_size+1))
print(list(x.as_numpy_iterator())[:2], '<- first two window after batch with 9')

x = x.map(lambda x: (x[:-1], x[-1]))
print(list(x.as_numpy_iterator())[:4])
